In [0]:
import os
import openai
import logging
import dataiku
import resource
import pandas as pd

from time import time
from utils_prompt import prepare_excel_for_pdf, convert_office_to_pdf, extract_image_pdf

In [0]:
logging.getLogger('openai').setLevel(logging.WARNING)
logging.getLogger('httpx').setLevel(logging.WARNING)

client = dataiku.api_client()
auth_info = client.get_auth_info(with_secrets=True)

for secret in auth_info['secrets']:
    if secret['key'] == 'neura-gpt-test-key':
        API_KEY = secret['value']
        break
        
openai.api_type = 'azure'
openai.azure_endpoint = 'https://iapi-test.merck.com/gpt/libsupport'
openai.api_version = '2025-03-01-preview'
openai.api_key = API_KEY

model_name = 'gpt-4o-2024-11-20'
price_per_mil_token = (2.5, 10)

print("test")

In [0]:
folder_pref = '/tmp/auto_soe'

environ = os.environ.copy()
environ['XDG_RUNTIME_DIR'] = folder_pref

os.makedirs(folder_pref, exist_ok=True)

sources = [
    dataiku.Folder('jpYSb1ya'),
    dataiku.Folder('Ms4f1qz8')
]

all_entries = []
for fobj in sources:
    for p in fobj.list_paths_in_partition():
        all_entries.append((fobj, p))
            
print(all_entries)      


for folder_obj, path in all_entries:
    fid = getattr(folder_obj, "get_id", lambda: None)() or f"obj_{hex(id(folder_obj))}"
    raw = path
    norm = raw.lstrip('/')

    print("=== ENTRY ===")
    print("folder id:", fid)
    print("raw path repr:", repr(raw))
    print("normalized path repr:", repr(norm))





In [0]:
## Perform the extraction
price_total = 0
tokens_total = 0
output_list = []


for folder_obj, path in all_entries:
    basename = os.path.basename(p)
    dirname = os.path.dirname(p)
    

    output_row = {
        'Dir_Name': dirname,
        'File_Name': basename,
        'Chunk_Id': 0,
        'Chunk_Text': '',
        'Chunk_Length': 0,
        'Token_Count': 0
    }

    file_size = folder_obj.get_path_details(p)['size'] / (1024 * 1024)
    memory_usage = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024
    
    log_row = {'directory': dirname,
               'file_name': basename,
               'file_size':  f'{file_size:.2f} MB', 
               'memory_total': f'{memory_usage:.2f} MB',
               'page_count': '', 
               'token_count': '', 
               'extract_duration': '',
               'price_total': '',
               'error': ''}
